# Scraping data Telegram

### Data collection

# Website for scraping (Ukraine)
https://t.me/kyivoperat             https://web.telegram.org/a/#-1001061098805

https://t.me/UaOnlii                https://web.telegram.org/a/#-1001233777422

https://t.me/DeepStateUA            https://web.telegram.org/a/#-1001469021333

https://t.me/novinach               https://web.telegram.org/k/#@novinach

# Website for scraping (Russia)

https://t.me/boris_rozhin           https://web.telegram.org/a/#-1001101806611
 
https://t.me/dva_majors             https://web.telegram.org/a/#-1001513431778

https://t.me/readovkanews           https://web.telegram.org/a/#-1001260622817

In [1]:
import pandas as pd
from telethon.sync import TelegramClient
from telethon.tl.functions.messages import GetDiscussionMessageRequest
from datetime import datetime, timezone

In [2]:
api_id = "YOUR_APP_ID"
api_hash = "YOUR_APP_HASH"

Credentials removed for security. To run this notebook, you need your own app_id and app_hash from https://my.telegram.org

# Scraping Ukrainian Telegram Channels

# ✙ Київ Оперативний | Kyiv Operative ✙

In [4]:
# Creating the client session
client = TelegramClient("my_session", api_id, api_hash)
await client.start()

# Public channel ID 
channel_id_ko = -1001061098805
entity_ko = await client.get_entity(channel_id_ko)
channel_username = entity_ko.username if hasattr(entity_ko, "username") else "unknown"

# Defining date range
start_date = datetime(2025, 2, 14, tzinfo=timezone.utc)
end_date = datetime(2025, 3, 14, tzinfo=timezone.utc)

# Collecting messages
messages = []
async for message in client.iter_messages(
    entity_ko,
    offset_date=end_date   # messages **before this date**
):
    if message.date < start_date:
        break
    if start_date <= message.date <= end_date:
        messages.append({
            "post_id": message.id,
            "date": message.date,
            "text": message.text,
            "views": message.views,
            "forwards": message.forwards,
            "channel_id": channel_id_ko,
            "channel_name": channel_username,
        })

# Converting to DataFrame
df_ko = pd.DataFrame(messages)
df_ko

,post_id,date,text,views,forwards,channel_id,channel_name
0,166347,2025-03-13 20:27:58+00:00,На перехресті Великої Житомирської та Володими...,70052.0,42.0,-1001061098805,kyivoperat
1,166345,2025-03-13 19:42:49+00:00,Зараз ми всі почули з Росії дуже передбачувані...,67290.0,14.0,-1001061098805,kyivoperat
2,166344,2025-03-13 18:47:06+00:00,Тільки сьогодні 12-річний зачепер у Соломʼянсь...,64979.0,278.0,-1001061098805,kyivoperat
3,166343,2025-03-13 18:25:06+00:00,Просто водій білого «пиріжка» виїжджає на голо...,61272.0,22.0,-1001061098805,kyivoperat
4,166340,2025-03-13 17:34:22+00:00,Багато хто питав у коментарях що саме знімала ...,61219.0,237.0,-1001061098805,kyivoperat
...,...,...,...,...,...,...,...
1536,164697,2025-02-14 06:19:52+00:00,,54660.0,129.0,-1001061098805,kyivoperat
1537,164696,2025-02-14 06:19:52+00:00,,55270.0,130.0,-1001061098805,kyivoperat
1538,164695,2025-02-14 06:19:52+00:00,,54118.0,129.0,-1001061098805,kyivoperat
1539,164694,2025-02-14 06:19:52+00:00,‼️**Трагедія на річці Інгулець: тривають пошук...,53414.0,129.0,-1001061098805,kyivoperat


In [5]:
df_ko.to_csv("data/ua/Posts_ko.csv", index=False)

Retrieving comments for the channel "Kyiv Operative"

In [5]:
async def get_comments():
    comments_ko = []

    for _, row in df_ko.iterrows():
        post_id = row["post_id"]

        try:
            msg = await client.get_messages(entity_ko, ids=post_id)
            if not msg or not msg.replies or not msg.replies.replies:
                print(f"Post {post_id} has no discussion.")
                continue

            discussion = await client(GetDiscussionMessageRequest(
                peer=entity_ko,
                msg_id=post_id
            ))

            discussion_msg = discussion.messages[0]
            discussion_chat = discussion.chats[0]

            async for reply in client.iter_messages(
                discussion_chat.id,
                reply_to=discussion_msg.id
            ):
                if reply.text:
                    comments_ko.append({
                        "post_id": post_id,
                        "comment_text": reply.text,
                        "comment_date": reply.date,
                        "comment_user_id": reply.sender_id
                    })

        except Exception as e:
            print(f"Failed to get comments for post {post_id}: {e}")

    return pd.DataFrame(comments_ko)

In [6]:
df_comments_ko = await get_comments()
df_comments_ko.head()

Post 166340 has no discussion.
Failed to get comments for post 166339: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 166335 has no discussion.
Post 166334 has no discussion.
Failed to get comments for post 166333: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 166327 has no discussion.
Post 166326 has no discussion.
Failed to get comments for post 166325: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 166324 has no discussion.
Failed to get comments for post 166323: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 166322 has no discussion.
Post 166321 has no discussion.
Failed to get comments for post 166320: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 166312 has no discussion.
Post 166297 has no discussion.
Failed to get comments for post 166296: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 1

Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup
Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup
Server resent the older message 7515520623706737665, ignoring
Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 165537 has no discussion.
Failed to get comments for post 165536: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 165535 has no discussion.
Post 165534 has no discussion.
Post 165533 has no discussion.
Failed to get comments for post 165532: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 165531 has no discussion.
Post 165530 has no discussion.
Post 165529 has no discussion.
Post 165528 has no discussion.
Failed to get comments for post 165527: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 165526 has no discussion.


Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 165522 has no discussion.
Post 165521 has no discussion.
Post 165520 has no discussion.
Post 165519 has no discussion.
Post 165518 has no discussion.


...

Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 164760 has no discussion.
Post 164754 has no discussion.
Post 164753 has no discussion.
Failed to get comments for post 164752: The message ID used in the peer was invalid (caused by GetRepliesRequest)


Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 164736 has no discussion.
Post 164735 has no discussion.
Post 164734 has no discussion.
Failed to get comments for post 164733: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 164730 has no discussion.
Failed to get comments for post 164729: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 164728 has no discussion.
Post 164727 has no discussion.
Post 164726 has no discussion.
Post 164725 has no discussion.
Post 164724 has no discussion.
Post 164723 has no discussion.
Post 164722 has no discussion.
Post 164721 has no discussion.
Failed to get comments for post 164720: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 164714 has no discussion.
Post 164713 has no discussion.
Post 164712 has no discussion.
Post 164711 has no discussion.
Post 164710 has no discussion.
Failed to get comments for post 164709: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 164706 has n

,post_id,comment_text,comment_date,comment_user_id
0,166347,Хтось мчав на помаранчевий колір світлофора?,2025-03-14 06:59:44+00:00,5.147544e+08
1,166347,Ну що за проблема зробити кругове???,2025-03-14 04:51:26+00:00,6.131473e+09
2,166347,"Ааа, ну тоді най з ним) кістка в язиці, то так...",2025-03-14 04:01:20+00:00,4.888828e+08
3,166347,Женщіна вона всюди женщіна 😄😄,2025-03-13 21:58:47+00:00,9.873702e+08
4,166347,Оно такое смешное и дурное)) кошки обычно дики...,2025-03-13 21:58:01+00:00,5.194757e+09


In [7]:
df_comments_ko.to_csv("data/ua/Comments_ko.csv", index=False)

Doing the same with all the following channels

# ✙DeepState✙🇺🇦

In [3]:
# Creating the client session
client = TelegramClient("my_session", api_id, api_hash)
await client.start()

# Public channel ID 
channel_id_ds= -1001469021333
entity_ds = await client.get_entity(channel_id_ds)
channel_username = entity_ds.username if hasattr(entity_ds, "username") else "unknown"

# Defining date range
start_date = datetime(2025, 2, 14, tzinfo=timezone.utc)
end_date = datetime(2025, 3, 14, tzinfo=timezone.utc)

# Collecting messages
messages = []
async for message in client.iter_messages(
        entity_ds,
        offset_date=end_date  # messages **before this date**
):
    if message.date < start_date:
        break
    if start_date <= message.date <= end_date:
        messages.append({
            "post_id": message.id,
            "date": message.date,
            "text": message.text,
            "views": message.views,
            "forwards": message.forwards,
            "channel_id": channel_id_ds,
            "channel_name": channel_username,
        })

# Converting to DataFrame
df_ds = pd.DataFrame(messages)
df_ds

,post_id,date,text,views,forwards,channel_id,channel_name
0,21462,2025-03-13 21:41:55+00:00,**🌊**** **[**36 ОБрМП**](https://t.me/ua_marin...,311822,370,-1001469021333,DeepStateUA
1,21461,2025-03-13 18:46:49+00:00,**🤝 Повністю згодні з думкою наших друзів з РН...,302309,280,-1001469021333,DeepStateUA
2,21460,2025-03-13 17:03:33+00:00,⚖️ **ЄСПЛ **[**визнав**](https://hudoc.echr.co...,335154,896,-1001469021333,DeepStateUA
3,21459,2025-03-13 13:48:11+00:00,**🔄**** Мапу оновлено\n**\n⚔️ Ворог просунувся...,296881,154,-1001469021333,DeepStateUA
4,21458,2025-03-13 12:20:54+00:00,**🟡****Кадри ураження кацапні в Басівці від бі...,371653,557,-1001469021333,DeepStateUA
...,...,...,...,...,...,...,...
175,21283,2025-02-15 07:38:48+00:00,**🇩🇪**** Медіана опитувань перед Парламентськи...,262553,441,-1001469021333,DeepStateUA
176,21282,2025-02-14 23:45:47+00:00,**🔄**** Мапу оновлено\n**\n⚔️ Ворог просунувся...,272033,115,-1001469021333,DeepStateUA
177,21281,2025-02-14 12:58:01+00:00,🇷🇺 **Просування кацапів на правому березі річк...,292437,743,-1001469021333,DeepStateUA
178,21280,2025-02-14 12:36:50+00:00,**🕯 В Україну повернули тіла 757 українських Г...,274636,329,-1001469021333,DeepStateUA


In [4]:
df_ds.to_csv("data/ua/Posts_ds.csv", index=False)

In [13]:
async def get_comments():
    comments_ds = []

    for _, row in df_ds.iterrows():
        post_id = row["post_id"]

        try:
            msg = await client.get_messages(entity_ds, ids=post_id)
            if not msg or not msg.replies or not msg.replies.replies:
                print(f"Post {post_id} has no discussion.")
                continue

            discussion = await client(GetDiscussionMessageRequest(
                peer=entity_ds,
                msg_id=post_id
            ))

            discussion_msg = discussion.messages[0]
            discussion_chat = discussion.chats[0]

            async for reply in client.iter_messages(
                discussion_chat.id,
                reply_to=discussion_msg.id
            ):
                if reply.text:
                    comments_ds.append({
                        "post_id": post_id,
                        "comment_text": reply.text,
                        "comment_date": reply.date,
                        "comment_user_id": reply.sender_id
                    })

        except Exception as e:
            print(f"Failed to get comments for post {post_id}: {e}")

    return pd.DataFrame(comments_ds)

In [14]:
df_comments_ds = await get_comments()
df_comments_ds.head()

Post 21429 has no discussion.
Post 21403 has no discussion.
Failed to get comments for post 21402: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 21393 has no discussion.
Post 21374 has no discussion.


Cannot get difference since the account is likely misusing the session: database is locked
Unhandled exception from keepalive_handle after cancelling <class '_asyncio.Task'> (<Task finished name='Task-12' coro=<UpdateMethods._keepalive_loop() done, defined at /Applications/anaconda3/lib/python3.11/site-packages/telethon/client/updates.py:481> exception=OperationalError('database is locked')>)
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.11/site-packages/telethon/helpers.py", line 176, in _cancel
    await task
  File "/Applications/anaconda3/lib/python3.11/site-packages/telethon/client/updates.py", line 517, in _keepalive_loop
    self._save_states_and_entities()
  File "/Applications/anaconda3/lib/python3.11/site-packages/telethon/client/telegrambaseclient.py", line 694, in _save_states_and_entities
    self.session.process_entities(types.contacts.ResolvedPeer(None, [e._as_input_peer() for e in entities], []))
  File "/Applications/anaconda3/lib/pytho

Post 21364 has no discussion.
Post 21335 has no discussion.
Post 21321 has no discussion.
Post 21309 has no discussion.


,post_id,comment_text,comment_date,comment_user_id
0,21462,головне що кацапи дохнуть в муках. а ті кацапи...,2025-03-14 09:27:40+00:00,6.189523e+09
1,21462,у кожного може бути своя думка... без проблем,2025-03-14 06:58:47+00:00,6.763106e+09
2,21462,ну ти майже такий самий стратег як радіоактивн...,2025-03-14 06:57:49+00:00,8.454073e+08
3,21462,"так, саме так і думаю .",2025-03-14 06:56:22+00:00,6.763106e+09
4,21462,"Люди які кажуть, що росія зайшла на Сумщину ч...",2025-03-14 06:49:36+00:00,9.346062e+08


In [15]:
df_comments_ds.to_csv("data/ua/Comments_ds.csv", index=False)

# Україна Online: Новини | Політика

In [3]:
# Creating the client session
client = TelegramClient("my_session", api_id, api_hash)
await client.start()

# Public channel ID 
channel_id_uo= -1001233777422
entity_uo = await client.get_entity(channel_id_uo)
channel_username = entity_uo.username if hasattr(entity_uo, "username") else "unknown"

# Defining date range
start_date = datetime(2025, 2, 14, tzinfo=timezone.utc)
end_date = datetime(2025, 3, 14, tzinfo=timezone.utc)

# Collecting messages
messages = []
async for message in client.iter_messages(
        entity_uo,
        offset_date=end_date  # messages **before this date**
):
    if message.date < start_date:
        break
    if start_date <= message.date <= end_date:
        messages.append({
            "post_id": message.id,
            "date": message.date,
            "text": message.text,
            "views": message.views,
            "forwards": message.forwards,
            "channel_id": channel_id_uo,
            "channel_name": channel_username,
        })

# Converting to DataFrame
df_uo = pd.DataFrame(messages)
df_uo

,post_id,date,text,views,forwards,channel_id,channel_name
0,97026,2025-03-13 21:51:13+00:00,❗️**Трамп не вводив нових санкцій проти рф: їх...,303971,652,-1001233777422,UaOnlii
1,97025,2025-03-13 21:37:21+00:00,**❗️Завдяки діям Трампа майже всі великі росій...,292042,934,-1001233777422,UaOnlii
2,97024,2025-03-13 21:24:05+00:00,**❗️Спецпредставника США Келлога відсторонили ...,276603,688,-1001233777422,UaOnlii
3,97023,2025-03-13 21:13:59+00:00,**❗️США відновлюють постачання Україні далекоб...,274636,656,-1001233777422,UaOnlii
4,97022,2025-03-13 21:10:44+00:00,**❗️Тиша протрималася недовго: відбулися пуски...,254215,241,-1001233777422,UaOnlii
...,...,...,...,...,...,...,...
2090,94787,2025-02-14 07:17:53+00:00,"**❗️Вночі дрон рф атакував Чорнобильську АЕС, ...",231312,2596,-1001233777422,UaOnlii
2091,94786,2025-02-14 06:40:39+00:00,**Не ведіться на емоційні гойдалки. Інтереси У...,218032,323,-1001233777422,UaOnlii
2092,94785,2025-02-14 06:39:25+00:00,**Ціни на ліки офіційно знизять на 30% з 1 бер...,243144,256,-1001233777422,UaOnlii
2093,94784,2025-02-14 06:01:15+00:00,"**❗️Трамп передасть Україні ядерну зброю, якщо...",258946,1286,-1001233777422,UaOnlii


In [4]:
df_uo.to_csv("data/ua/Posts_uo.csv", index=False)

In [6]:
async def get_comments():
    comments_uo = []

    for _, row in df_uo.iterrows():
        post_id = row["post_id"]

        try:
            msg = await client.get_messages(entity_uo, ids=post_id)
            if not msg or not msg.replies or not msg.replies.replies:
                print(f"Post {post_id} has no discussion.")
                continue

            discussion = await client(GetDiscussionMessageRequest(
                peer=entity_uo,
                msg_id=post_id
            ))

            discussion_msg = discussion.messages[0]
            discussion_chat = discussion.chats[0]

            async for reply in client.iter_messages(
                discussion_chat.id,
                reply_to=discussion_msg.id
            ):
                if reply.text:
                    comments_uo.append({
                        "post_id": post_id,
                        "comment_text": reply.text,
                        "comment_date": reply.date,
                        "comment_user_id": reply.sender_id
                    })

        except Exception as e:
            print(f"Failed to get comments for post {post_id}: {e}")

    return pd.DataFrame(comments_uo)

In [7]:
df_comments_uo = await get_comments()
df_comments_uo.head()

Post 97006 has no discussion.
Post 97005 has no discussion.
Post 97004 has no discussion.
Post 96972 has no discussion.
Failed to get comments for post 96971: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 96964 has no discussion.
Failed to get comments for post 96963: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 96941 has no discussion.
Post 96940 has no discussion.
Post 96921 has no discussion.
Post 96919 has no discussion.
Post 96918 has no discussion.
Post 96908 has no discussion.
Failed to get comments for post 96907: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 96906 has no discussion.
Post 96905 has no discussion.
Failed to get comments for post 96904: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 96880 has no discussion.
Failed to get comments for post 96879: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 96878 has 

Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup
Server resent the older message 7516967791634007041, ignoring
Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup
Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 96671 has no discussion.
Failed to get comments for post 96670: The message ID used in the peer was invalid (caused by GetRepliesRequest)


Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 96665 has no discussion.
Failed to get comments for post 96664: The message ID used in the peer was invalid (caused by GetRepliesRequest)


...

Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 95141 has no discussion.
Failed to get comments for post 95140: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 95137 has no discussion.


Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 95131 has no discussion.
Post 95130 has no discussion.
Failed to get comments for post 95129: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 95094 has no discussion.
Failed to get comments for post 95093: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 95092 has no discussion.
Post 95039 has no discussion.
Post 95038 has no discussion.
Failed to get comments for post 95037: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 95017 has no discussion.
Failed to get comments for post 95016: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 95008 has no discussion.
Post 95007 has no discussion.
Failed to get comments for post 95006: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 94991 has no discussion.
Failed to get comments for post 94990: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 94976 has no disc

,post_id,comment_text,comment_date,comment_user_id
0,97026,"Це ти пиздиш, бо дурний, як би був розумний, т...",2025-03-14 08:49:03+00:00,1.303107e+09
1,97026,Я в 30 км від лбз.Мені розказувать нічого не т...,2025-03-14 07:54:34+00:00,7.624276e+08
2,97026,"Якщо ти в окопі, то мабуть думаєш прийдуть рос...",2025-03-14 07:39:37+00:00,1.303107e+09
3,97026,"І щоб ти, Валєра у окопі також був!Бо без тебе...",2025-03-14 06:36:28+00:00,7.624276e+08
4,97026,"23:37, ...23:54 ))))",2025-03-14 05:46:31+00:00,2.553465e+08


In [8]:
df_comments_uo.to_csv("data/ua/Comments_uo.csv", index=False)

# Новинач

In [ ]:
# Creating the client session
client = TelegramClient("my_session", api_id, api_hash)
await client.start()

# Public channel ID 
channel_id_no = -1001134948258
entity_no = await client.get_entity(channel_id_no)
channel_username = entity_no.username if hasattr(entity_no, "username") else "unknown"

# Defining date range
start_date = datetime(2025, 2, 14, tzinfo=timezone.utc)
end_date = datetime(2025, 3, 14, tzinfo=timezone.utc)

# Collect messages
messages = []
async for message in client.iter_messages(
    entity_no,
    offset_date=end_date   # messages **before this date**
):
    if message.date < start_date:
        break
    if start_date <= message.date <= end_date:
        messages.append({
            "post_id": message.id,
            "date": message.date,
            "text": message.text,
            "views": message.views,
            "forwards": message.forwards,
            "channel_id": channel_id_no,
            "channel_name": channel_username,
        })

# Converting to DataFrame
df_no = pd.DataFrame(messages)
df_no

In [ ]:
df_no.to_csv("data/ua/Posts_no.csv", index=False)

In [ ]:
async def get_comments():
    comments_no = []

    for _, row in df_no.iterrows():
        post_id = row["post_id"]

        try:
            msg = await client.get_messages(entity_no, ids=post_id)
            if not msg or not msg.replies or not msg.replies.replies:
                print(f"Post {post_id} has no discussion.")
                continue

            discussion = await client(GetDiscussionMessageRequest(
                peer=entity_no,
                msg_id=post_id
            ))

            discussion_msg = discussion.messages[0]
            discussion_chat = discussion.chats[0]

            async for reply in client.iter_messages(
                    discussion_chat.id,
                    reply_to=discussion_msg.id
            ):
                if reply.text:
                    comments_no.append({
                        "post_id": post_id,
                        "comment_text": reply.text,
                        "comment_date": reply.date,
                        "comment_user_id": reply.sender_id
                    })

        except Exception as e:
            print(f"Failed to get comments for post {post_id}: {e}")

    return pd.DataFrame(comments_no)


In [ ]:
df_comments_no = await get_comments()
df_comments_no.head()

In [ ]:
df_comments_no.to_csv("data/ua/Comments_no.csv", index=False)

# Scraping Russian Telegram Channels

# Colonelcassad

In [3]:
# Creating the client session
client = TelegramClient("my_session", api_id, api_hash)
await client.start()

# Public channel ID 
channel_id_cl= -1001101806611
entity_cl = await client.get_entity(channel_id_cl)
channel_username = entity_cl.username if hasattr(entity_cl, "username") else "unknown"

# Defining date range
start_date = datetime(2025, 2, 14, tzinfo=timezone.utc)
end_date = datetime(2025, 3, 14, tzinfo=timezone.utc)

# Collecting messages
messages = []
async for message in client.iter_messages(
        entity_cl,
        offset_date=end_date  # messages **before this date**
):
    if message.date < start_date:
        break
    if start_date <= message.date <= end_date:
        messages.append({
            "post_id": message.id,
            "date": message.date,
            "text": message.text,
            "views": message.views,
            "forwards": message.forwards,
            "channel_id": channel_id_cl,
            "channel_name": channel_username,
        })

# Converting to DataFrame
df_cl = pd.DataFrame(messages)
df_cl

,post_id,date,text,views,forwards,channel_id,channel_name
0,157779,2025-03-13 23:20:52+00:00,"О тех, кто занимался военными преступлениями н...",253245.0,210.0,-1001101806611,boris_rozhin
1,157778,2025-03-13 23:12:01+00:00,[Операторы](https://t.me/mod_russia/50058) FPV...,246294.0,56.0,-1001101806611,boris_rozhin
2,157777,2025-03-13 22:23:01+00:00,**Европейский Союз очень и очень противный \nЯ...,250403.0,492.0,-1001101806611,boris_rozhin
3,157776,2025-03-13 22:11:29+00:00,,248492.0,505.0,-1001101806611,boris_rozhin
4,157775,2025-03-13 22:11:29+00:00,**Очередная самоделка с Покровского направлени...,254333.0,505.0,-1001101806611,boris_rozhin
...,...,...,...,...,...,...,...
3208,154522,2025-02-14 00:03:01+00:00,,184352.0,43.0,-1001101806611,boris_rozhin
3209,154521,2025-02-14 00:03:01+00:00,,184669.0,43.0,-1001101806611,boris_rozhin
3210,154520,2025-02-14 00:03:01+00:00,,185291.0,44.0,-1001101806611,boris_rozhin
3211,154519,2025-02-14 00:03:01+00:00,,183723.0,43.0,-1001101806611,boris_rozhin


In [4]:
df_cl.to_csv("data/ru/Posts_cl.csv", index=False) 

In [8]:
async def get_comments():
    comments_cl = []

    for _, row in df_cl.iterrows():
        post_id = row["post_id"]

        try:
            msg = await client.get_messages(entity_cl, ids=post_id)
            if not msg or not msg.replies or not msg.replies.replies:
                print(f"Post {post_id} has no discussion.")
                continue

            discussion = await client(GetDiscussionMessageRequest(
                peer=entity_cl,
                msg_id=post_id
            ))

            discussion_msg = discussion.messages[0]
            discussion_chat = discussion.chats[0]

            async for reply in client.iter_messages(
                discussion_chat.id,
                reply_to=discussion_msg.id
            ):
                if reply.text:
                    comments_cl.append({
                        "post_id": post_id,
                        "comment_text": reply.text,
                        "comment_date": reply.date,
                        "comment_user_id": reply.sender_id
                    })

        except Exception as e:
            print(f"Failed to get comments for post {post_id}: {e}")

    return pd.DataFrame(comments_cl)

In [9]:
df_comments_cl = await get_comments()
df_comments_cl.head()

Post 157776 has no discussion.
Failed to get comments for post 157775: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157767 has no discussion.
Failed to get comments for post 157766: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157755 has no discussion.
Failed to get comments for post 157754: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157741 has no discussion.
Post 157738 has no discussion.
Post 157737 has no discussion.
Post 157736 has no discussion.
Post 157735 has no discussion.
Post 157734 has no discussion.
Failed to get comments for post 157733: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157716 has no discussion.
Post 157715 has no discussion.
Post 157714 has no discussion.
Post 157713 has no discussion.
Post 157712 has no discussion.
Post 157711 has no discussion.
Failed to get comments for post 157710: The message ID used in the peer was inv

Cannot get difference since the account is likely misusing the session: database is locked
Unhandled exception from keepalive_handle after cancelling <class '_asyncio.Task'> (<Task finished name='Task-12' coro=<UpdateMethods._keepalive_loop() done, defined at /Applications/anaconda3/lib/python3.11/site-packages/telethon/client/updates.py:481> exception=OperationalError('database is locked')>)
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.11/site-packages/telethon/helpers.py", line 176, in _cancel
    await task
  File "/Applications/anaconda3/lib/python3.11/site-packages/telethon/client/updates.py", line 517, in _keepalive_loop
    self._save_states_and_entities()
  File "/Applications/anaconda3/lib/python3.11/site-packages/telethon/client/telegrambaseclient.py", line 694, in _save_states_and_entities
    self.session.process_entities(types.contacts.ResolvedPeer(None, [e._as_input_peer() for e in entities], []))
  File "/Applications/anaconda3/lib/pytho

Post 157677 has no discussion.
Post 157676 has no discussion.
Post 157675 has no discussion.
Post 157674 has no discussion.
Post 157673 has no discussion.
Failed to get comments for post 157672: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157654 has no discussion.
Failed to get comments for post 157653: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157643 has no discussion.
Post 157642 has no discussion.
Post 157641 has no discussion.
Post 157640 has no discussion.
Post 157639 has no discussion.
Post 157638 has no discussion.
Failed to get comments for post 157637: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157634 has no discussion.
Failed to get comments for post 157633: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157631 has no discussion.
Failed to get comments for post 157630: The message ID used in the peer was invalid (caused by GetRepliesReque

Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 157116 has no discussion.
Failed to get comments for post 157115: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157112 has no discussion.
Post 157111 has no discussion.
Post 157110 has no discussion.
Post 157109 has no discussion.
Post 157108 has no discussion.
Failed to get comments for post 157107: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157104 has no discussion.
Failed to get comments for post 157103: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157095 has no discussion.
Post 157094 has no discussion.
Post 157093 has no discussion.
Post 157092 has no discussion.
Post 157091 has no discussion.
Post 157090 has no discussion.
Post 157089 has no discussion.
Post 157088 has no discussion.
Post 157087 has no discussion.
Failed to get comments for post 157086: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 157076 has no discussion.
Post 157075 has n

,post_id,comment_text,comment_date,comment_user_id
0,157779,"теперь укры будут отбывть свое наказание, или ...",2025-03-14 14:14:01+00:00,7.498169e+09
1,157779,"Президент сказал ,что все кто попал в плен в К...",2025-03-14 10:44:48+00:00,5.336044e+09
2,157779,Правильно! Никто не останется безнаказанным. З...,2025-03-14 10:35:26+00:00,6.317807e+09
3,157779,"ну вот каклам тоже это все вернется, может не ...",2025-03-14 07:20:11+00:00,6.185647e+09
4,157779,ни один укронацист не избежит наказания за то ...,2025-03-14 06:56:22+00:00,6.163684e+09


In [10]:
df_comments_cl.to_csv("data/ru/Comments_cl.csv", index=False)

# Два майора

In [10]:
# Creating the client session
client = TelegramClient("my_session", api_id, api_hash)
await client.start()

# Public channel ID 
channel_id_dm= -1001513431778
entity_dm = await client.get_entity(channel_id_dm)
channel_username = entity_dm.username if hasattr(entity_dm, "username") else "unknown"

# Defining date range
start_date = datetime(2025, 2, 14, tzinfo=timezone.utc)
end_date = datetime(2025, 3, 14, tzinfo=timezone.utc)

# Collecting messages
messages = []
async for message in client.iter_messages(
        entity_dm,
        offset_date=end_date  # messages **before this date**
):
    if message.date < start_date:
        break
    if start_date <= message.date <= end_date:
        messages.append({
            "post_id": message.id,
            "date": message.date,
            "text": message.text,
            "views": message.views,
            "forwards": message.forwards,
            "channel_id": channel_id_dm,
            "channel_name": channel_username,
        })

# Converting to DataFrame
df_dm = pd.DataFrame(messages)
df_dm

Task was destroyed but it is pending!
task: <Task pending name='Task-32' coro=<Connection._send_loop() running at /Applications/anaconda3/lib/python3.11/site-packages/telethon/network/connection/connection.py:322> wait_for=<Future pending cb=[Task.task_wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-33' coro=<Connection._recv_loop() running at /Applications/anaconda3/lib/python3.11/site-packages/telethon/network/connection/connection.py:341> wait_for=<Future pending cb=[Task.task_wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-34' coro=<MTProtoSender._send_loop() running at /Applications/anaconda3/lib/python3.11/site-packages/telethon/network/mtprotosender.py:464> wait_for=<Future pending cb=[Task.task_wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-35' coro=<MTProtoSender._recv_loop() running at /Applications/anaconda3/lib/python3.11/site-packages/telethon/network/mtprotosender.py:507> w

,post_id,date,text,views,forwards,channel_id,channel_name
0,66717,2025-03-13 20:03:47+00:00,«…Коль в верхах большие лица\nНе нашли для мир...,352222.0,275.0,-1001513431778,dva_majors
1,66716,2025-03-13 19:50:18+00:00,**✨**** Сбор в** **Курскую область! ****✨****\...,553973.0,56.0,-1001513431778,dva_majors
2,66715,2025-03-13 19:40:59+00:00,"**Курская область. Суджа**, **кадры **[**Групп...",418278.0,1029.0,-1001513431778,dva_majors
3,66714,2025-03-13 19:08:55+00:00,[**Информация**](https://t.me/dva_majors/66712...,405044.0,4466.0,-1001513431778,dva_majors
4,66713,2025-03-13 19:01:43+00:00,,305629.0,5771.0,-1001513431778,dva_majors
...,...,...,...,...,...,...,...
2170,64517,2025-02-14 06:08:24+00:00,Началась подготовка к продвижению Залужного в ...,314479.0,1330.0,-1001513431778,dva_majors
2171,64516,2025-02-14 04:31:56+00:00,**Переговорный **[**процесс**](https://t.me/ne...,318387.0,590.0,-1001513431778,dva_majors
2172,64515,2025-02-14 04:20:56+00:00,**Минобороны России:**\n\nВ течение прошедшей ...,306357.0,240.0,-1001513431778,dva_majors
2173,64514,2025-02-14 03:43:50+00:00,**#Сводка**** на утро 14 февраля 2025 года**\n...,629203.0,853.0,-1001513431778,dva_majors


In [11]:
df_dm.to_csv("data/ru/Posts_dm.csv", index=False) 

In [5]:
async def get_comments():
    comments_dm = []

    for _, row in df_dm.iterrows():
        post_id = row["post_id"]

        try:
            msg = await client.get_messages(entity_dm, ids=post_id)
            if not msg or not msg.replies or not msg.replies.replies:
                print(f"Post {post_id} has no discussion.")
                continue

            discussion = await client(GetDiscussionMessageRequest(
                peer=entity_dm,
                msg_id=post_id
            ))

            discussion_msg = discussion.messages[0]
            discussion_chat = discussion.chats[0]

            async for reply in client.iter_messages(
                discussion_chat.id,
                reply_to=discussion_msg.id
            ):
                if reply.text:
                    comments_dm.append({
                        "post_id": post_id,
                        "comment_text": reply.text,
                        "comment_date": reply.date,
                        "comment_user_id": reply.sender_id
                    })

        except Exception as e:
            print(f"Failed to get comments for post {post_id}: {e}")

    return pd.DataFrame(comments_dm)

In [6]:
df_comments_dm = await get_comments()
df_comments_dm.head()

Post 66713 has no discussion.
Failed to get comments for post 66712: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 66706 has no discussion.
Post 66705 has no discussion.
Post 66704 has no discussion.
Failed to get comments for post 66703: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 66699 has no discussion.
Post 66698 has no discussion.
Post 66697 has no discussion.
Post 66696 has no discussion.
Post 66695 has no discussion.
Post 66694 has no discussion.
Post 66693 has no discussion.
Failed to get comments for post 66692: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 66682 has no discussion.
Post 66681 has no discussion.
Post 66679 has no discussion.
Post 66678 has no discussion.
Post 66677 has no discussion.
Post 66676 has no discussion.
Post 66675 has no discussion.
Post 66674 has no discussion.
Failed to get comments for post 66673: The message ID used in the peer was invalid (caused 

Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 66256 has no discussion.
Failed to get comments for post 66255: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 66247 has no discussion.


Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Failed to get comments for post 66246: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 66245 has no discussion.
Post 66244 has no discussion.
Post 66243 has no discussion.
Failed to get comments for post 66242: The message ID used in the peer was invalid (caused by GetRepliesRequest)


...

Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 65769 has no discussion.
Post 65768 has no discussion.
Post 65767 has no discussion.
Post 65766 has no discussion.
Post 65765 has no discussion.
Post 65764 has no discussion.
Post 65763 has no discussion.
Post 65762 has no discussion.
Post 65761 has no discussion.
Failed to get comments for post 65760: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 65759 has no discussion.
Post 65744 has no discussion.
Failed to get comments for post 65743: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 65736 has no discussion.
Failed to get comments for post 65735: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 65734 has no discussion.
Failed to get comments for post 65733: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 65728 has no discussion.
Post 65727 has no discussion.
Post 65726 has no discussion.
Post 65725 has no discussion.
Post 65724 has no discussion.
Post 657

Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup


Post 65713 has no discussion.
Post 65712 has no discussion.
Failed to get comments for post 65711: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 65692 has no discussion.
Post 65691 has no discussion.
Failed to get comments for post 65690: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 65682 has no discussion.
Post 65681 has no discussion.
Post 65680 has no discussion.
Post 65679 has no discussion.
Post 65678 has no discussion.
Post 65677 has no discussion.
Post 65676 has no discussion.
Post 65675 has no discussion.
Post 65674 has no discussion.
Failed to get comments for post 65673: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 65670 has no discussion.
Failed to get comments for post 65669: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 65668 has no discussion.
Post 65667 has no discussion.
Failed to get comments for post 65666: The message ID used in the pe

,post_id,comment_text,comment_date,comment_user_id
0,66717,Николай Анисимов абсолютно ПРАВ,2025-03-16 12:31:12+00:00,1.641041e+09
1,66717,нет,2025-03-14 15:37:39+00:00,6.177707e+09
2,66717,С ума сошли ?🤔,2025-03-14 15:23:11+00:00,5.762609e+09
3,66717,"Судя по тому, что вы влезаете в общение двух д...",2025-03-14 15:20:26+00:00,6.177707e+09
4,66717,"Судя по чему? 🙃\nВы ошиблись, всю жизнь живу в...",2025-03-14 15:18:44+00:00,5.762609e+09


In [7]:
df_comments_dm.to_csv("data/ru/Comments_dm.csv", index=False)

# Readovka

In [3]:
# Creating the client session
client = TelegramClient("my_session", api_id, api_hash)
await client.start()

# Public channel ID 
channel_id_re= -1001260622817
entity_re = await client.get_entity(channel_id_re)
channel_username = entity_re.username if hasattr(entity_re, "username") else "unknown"

# Defining date range
start_date = datetime(2025, 2, 14, tzinfo=timezone.utc)
end_date = datetime(2025, 3, 14, tzinfo=timezone.utc)

# Collecting messages
messages = []
async for message in client.iter_messages(
        entity_re,
        offset_date=end_date  # messages **before this date**
):
    if message.date < start_date:
        break
    if start_date <= message.date <= end_date:
        messages.append({
            "post_id": message.id,
            "date": message.date,
            "text": message.text,
            "views": message.views,
            "forwards": message.forwards,
            "channel_id": channel_id_re,
            "channel_name": channel_username,
        })

# Converting to DataFrame
df_re = pd.DataFrame(messages)
df_re

,post_id,date,text,views,forwards,channel_id,channel_name
0,94053,2025-03-13 19:29:58+00:00,**Владимир Путин дипломатично расставил акцент...,959162,1495,-1001260622817,readovkanews
1,94052,2025-03-13 18:59:58+00:00,**Подготовлены документы на снятие с розыска 1...,959836,925,-1001260622817,readovkanews
2,94051,2025-03-13 18:34:58+00:00,**ДУМ РФ и Турция снова признаются в любви дру...,1020364,1215,-1001260622817,readovkanews
3,94050,2025-03-13 18:34:58+00:00,,991208,2131,-1001260622817,readovkanews
4,94049,2025-03-13 17:59:58+00:00,**Русская армия формирует новый плацдарм за ка...,903210,675,-1001260622817,readovkanews
...,...,...,...,...,...,...,...
1034,93006,2025-02-14 07:37:59+00:00,**Накануне переговоров в Мюнхене Зеленский «шо...,334579,2377,-1001260622817,readovkanews
1035,93005,2025-02-14 07:22:17+00:00,**Тайна пирогов раскрыта — ртуть в булочки доб...,292128,2216,-1001260622817,readovkanews
1036,93004,2025-02-14 06:20:01+00:00,"**США могут отправить войска на Украину, если ...",307165,1787,-1001260622817,readovkanews
1037,93003,2025-02-14 05:30:07+00:00,"❗️**«Всем спасибо, президенту Владимиру Владим...",319996,1688,-1001260622817,readovkanews


In [4]:
df_re.to_csv("data/ru/Posts_re.csv", index=False) 

In [10]:
async def get_comments():
    comments_re = []

    for _, row in df_re.iterrows():
        post_id = row["post_id"]

        try:
            msg = await client.get_messages(entity_re, ids=post_id)
            if not msg or not msg.replies or not msg.replies.replies:
                print(f"Post {post_id} has no discussion.")
                continue

            discussion = await client(GetDiscussionMessageRequest(
                peer=entity_re,
                msg_id=post_id
            ))

            discussion_msg = discussion.messages[0]
            discussion_chat = discussion.chats[0]

            async for reply in client.iter_messages(
                discussion_chat.id,
                reply_to=discussion_msg.id
            ):
                if reply.text:
                    comments_re.append({
                        "post_id": post_id,
                        "comment_text": reply.text,
                        "comment_date": reply.date,
                        "comment_user_id": reply.sender_id
                    })

        except Exception as e:
            print(f"Failed to get comments for post {post_id}: {e}")

    return pd.DataFrame(comments_re)

In [11]:
df_comments_re = await get_comments()
df_comments_re.head()

Post 94051 has no discussion.
Failed to get comments for post 94050: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 94030 has no discussion.
Failed to get comments for post 94029: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 94027 has no discussion.
Failed to get comments for post 94026: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 94020 has no discussion.
Failed to get comments for post 94019: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 94013 has no discussion.
Failed to get comments for post 94012: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 94007 has no discussion.
Failed to get comments for post 94006: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 93994 has no discussion.
Post 93993 has no discussion.
Failed to get comments for post 93992: The message ID used in the peer was invalid

Cannot get difference since the account is likely misusing the session: database is locked
Unhandled exception from keepalive_handle after cancelling <class '_asyncio.Task'> (<Task finished name='Task-12' coro=<UpdateMethods._keepalive_loop() done, defined at /Applications/anaconda3/lib/python3.11/site-packages/telethon/client/updates.py:481> exception=OperationalError('database is locked')>)
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.11/site-packages/telethon/helpers.py", line 176, in _cancel
    await task
  File "/Applications/anaconda3/lib/python3.11/site-packages/telethon/client/updates.py", line 517, in _keepalive_loop
    self._save_states_and_entities()
  File "/Applications/anaconda3/lib/python3.11/site-packages/telethon/client/telegrambaseclient.py", line 694, in _save_states_and_entities
    self.session.process_entities(types.contacts.ResolvedPeer(None, [e._as_input_peer() for e in entities], []))
  File "/Applications/anaconda3/lib/pytho

Post 93952 has no discussion.
Failed to get comments for post 93951: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 93950 has no discussion.
Failed to get comments for post 93949: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 93942 has no discussion.
Failed to get comments for post 93941: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 93938 has no discussion.
Failed to get comments for post 93937: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 93933 has no discussion.
Failed to get comments for post 93932: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 93929 has no discussion.
Failed to get comments for post 93928: The message ID used in the peer was invalid (caused by GetRepliesRequest)
Post 93911 has no discussion.
Failed to get comments for post 93910: The message ID used in the peer was invalid (caused by GetRepliesRequest)

,post_id,comment_text,comment_date,comment_user_id
0,94053,Хохляцеих тварей надо уничтожить,2025-03-15 15:23:38+00:00,1755279905
1,94053,Какие гости! Это теперь общий дом для русских ...,2025-03-14 20:34:30+00:00,5925529639
2,94053,"Путин разумный - мир хорошо, но без оккупантов!",2025-03-14 19:56:09+00:00,7711235740
3,94053,А с Харьковской КАК? Оставить? Чтобы с границы...,2025-03-14 18:20:51+00:00,6649515897
4,94053,"Путин непрестанно повторяет, что России нужны ...",2025-03-14 16:49:52+00:00,7702598460


In [12]:
df_comments_re.to_csv("data/ru/Comments_re.csv", index=False)